In [4]:
# import libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import date
from dateutil.rrule import rrule, DAILY

import json

In [5]:
def crawler_boletins(dia, mes, ano):

    # Firefox WebDrive
    options = Options()
    options.headless = True    #Not open the broser 
    driver = webdriver.Firefox(options=options, executable_path = r'C:\Users\upe2\bin\geckodriver.exe')

    # Open CPTEC wepage
    driver.get("http://tempo.cptec.inpe.br/boletimtecnico/pt")

    #Get the CSS frame for select reports ans switch context
    frame = driver.find_element_by_css_selector('div.cont_dir iframe')
    driver.switch_to.frame(frame)

    #Select date and press enter
    elem = driver.find_element_by_id("j_id_jsp_31316922_1:dia")
    if dia != 1:
        for n in range(dia-1):
            elem.send_keys(Keys.ARROW_DOWN)

    elem = driver.find_element_by_id("j_id_jsp_31316922_1:mes")
    if mes != 1:
        for n in range(mes-1):
            elem.send_keys(Keys.ARROW_DOWN)

    elem = driver.find_element_by_id("j_id_jsp_31316922_1:ano")
    if ano != 2021:
        for n in range(2021 - ano):
            elem.send_keys(Keys.ARROW_DOWN)

    elem = driver.find_element_by_id("j_id_jsp_31316922_1:submit")
    elem.send_keys(Keys.RETURN)

    #Back the context to former frame
    driver.switch_to.parent_frame()

    #Get the CSS frame for select reports ans switch context
    frame = driver.find_element_by_css_selector('div.cont_meio iframe')
    driver.switch_to.frame(frame)
            
    #Convert dia, mes, ano in string
    # Dia
    if len(str(dia)) == 2:
        str_dia = str(dia)
    else:
        str_dia = "0" + str(dia)
    # Mes
    if len(str(mes)) == 2:
        str_mes = str(mes)
    else:
        str_mes = "0" + str(mes)

    #Ano
    str_ano = str(ano)
            
    try:
        
        element = WebDriverWait(driver, 3).until(
            EC.text_to_be_present_in_element((By.TAG_NAME, "em"), 
                                             "Análise Sinótica: " + str_dia + "/" + str_mes + "/" + str_ano + "-00Z")
                                                )
        
        #get the report texts
        elem = driver.find_elements_by_class_name("texto")
        texts = []
        for cell in elem:
            texts.append(cell.text)

        elem = driver.find_elements_by_class_name("sub")
        col = []
        for cell in elem:
            col.append(cell.text)
            
        # Transform lins in dictionary
        date_id = col[0].split(' ')[2]
        dic = {date_id: {col[1]:texts[1], 
                         col[2]:texts[2],
                         col[3]:texts[3],
                         col[4]:texts[4],
                         col[5]:texts[5]}}

        # Close CPTEC wepage
        driver.close()

        return(dic)
    
    except:
        driver.close()
        return ({str_dia + '/' + str_mes + '/' + str_ano + '-00Z' : {}})

In [20]:
# Start and finish date (YYYY, mm, dd)
start = date(2020, 1, 1)
finish = date(2020, 12, 31)

# Dictionary to store reports 
boletins = {}
file_name = 'Boletins_tecnicos_2020.json'
# Counter
n = 0   

for dt in rrule(DAILY, dtstart=start, until=finish):
    # Transform date into integer
    dia = int(dt.strftime("%d"))
    mes = int(dt.strftime("%m"))
    ano = int(dt.strftime("%Y"))
    
    # Get reports
    boletins.update(crawler_boletins(dia, mes, ano))
    n = n + 1
    
    # Save reports in JSON
    if n % 10 == 0:
        print (dt)
        with open(file_name, 'w') as fp:
            json.dump(boletins, fp)

with open(file_name, 'w') as fp:
    json.dump(boletins, fp)

2020-01-10 00:00:00
2020-01-20 00:00:00
2020-01-30 00:00:00
2020-02-09 00:00:00
2020-02-19 00:00:00
2020-02-29 00:00:00
2020-03-10 00:00:00
2020-03-20 00:00:00
2020-03-30 00:00:00
2020-04-09 00:00:00
2020-04-19 00:00:00
2020-04-29 00:00:00
2020-05-09 00:00:00
2020-05-19 00:00:00
2020-05-29 00:00:00
2020-06-08 00:00:00
2020-06-18 00:00:00
2020-06-28 00:00:00
2020-07-08 00:00:00
2020-07-18 00:00:00
2020-07-28 00:00:00
2020-08-07 00:00:00
2020-08-17 00:00:00
2020-08-27 00:00:00
2020-09-06 00:00:00
2020-09-16 00:00:00
2020-09-26 00:00:00
2020-10-06 00:00:00
2020-10-16 00:00:00
2020-10-26 00:00:00
2020-11-05 00:00:00
2020-11-15 00:00:00
2020-11-25 00:00:00
2020-12-05 00:00:00
2020-12-15 00:00:00
2020-12-25 00:00:00
